In [6]:
import json

import openai

client = openai.Client()

def saudacao_por_periodo(hora):
  if 5 <= hora < 12: return json.dumps({"saudacao": "Bom dia!"})
  elif 12 <= hora < 18: return json.dumps({"saudacao": "Bom dia!"})
  elif 18 <= hora < 23: return json.dumps({"saudacao": "Boa noite!"})
  else: return json.dumps({"saudacao": "Boa madrugada!"})

tools = [{
  "type": "function",
  "function":{
    "name": "saudacao_por_periodo",
    "parameters": {
      "type": "object",
      "properties": {
        "hora": {"type": "integer", "description": "A hora do dia em formato 24h"}
      },
      "required": ["hora"]
    }
  }
}]

funcao_disponivel = {
  "saudacao_por_periodo": saudacao_por_periodo
}

mensagens = [{"role": "user", "content": "Qual saudação o modelo me dá se for 23h?"}]

resposta = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=mensagens, tools=tools, tool_choice="auto")

msg_resp = resposta.choices[0].message
tool_calls = msg_resp.tool_calls
if tool_calls:
  mensagens.append(msg_resp)
  for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = funcao_disponivel[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(hora=function_args.get("hora"))

    mensagens.append({
      "tool_call_id": tool_call.id,
      "role": "tool",
      "name": function_name,
      "content": function_response

    })

  segunda_resposta = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=mensagens)

segunda_resposta.choices[0].message

ChatCompletionMessage(content='Boa madrugada!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [7]:
msg_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HJdCmXHjmk0RrlWHThzCeDeW', function=Function(arguments='{"hora":23}', name='saudacao_por_periodo'), type='function')])